In [4]:
import gmaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# Google developer API key
from Config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)


In [5]:
#create cordinates for NY State map
coordinates = [(42.9915, -76.0718)]

In [6]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(zoom_level = 6.5, center = (42.9915, -76.0718), layout = figure_layout, map_type="HYBRID")
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [7]:
#introducing Vehicle data
vehicle_df = pd.read_csv("vehicle_data.csv.gz", compression="gzip")

#take care of NaN velue
vehicle_df = vehicle_df.dropna()

#creating dataframe for car's for selective years
vehicle_year_df = vehicle_df.loc[vehicle_df['model_year'].astype(int) <= 2009]
#vehicle_year_df.head()

In [8]:
#calculating vehicle age till 2009 by model year 
veh_age = 2009 - vehicle_year_df['model_year'].astype(int)

#creating new data frame with new column for vehicle_age
new_vehicle_df = pd.DataFrame(vehicle_year_df)
new_vehicle_df['vehicle_age'] = veh_age
#new_vehicle_df.head()

In [9]:
#introducing zip code coordinates
zip_df = pd.read_csv('../Se7en/us-zip-code-latitude-and-longitude.csv')
zip_df = zip_df.rename (columns = {"Zip" : "zip"})

#take care of NaN velue
zip_df = zip_df.dropna()
#zip_df.head()

In [10]:
# find mean of vehicle age by zip code
veh_age_mean_df = new_vehicle_df.groupby('zip').agg({'vehicle_age':'mean'})
veh_age_mean_df = veh_age_mean_df.reset_index()
veh_age_mean_df.head()




,zip,vehicle_age
0,0,5.00
1,646,33.75
2,727,3.00
3,802,4.00
4,1002,0.50


In [11]:
#add zip code coordinates to vehicle age mean dataframe
vehicle_age_2009_df = pd.merge(veh_age_mean_df, zip_df, on=['zip','zip'], how="left")

#take care of NaN velue
vehicle_age_2009_df = vehicle_age_2009_df.dropna()
vehicle_age_2009_df

,zip,vehicle_age,City,State,Latitude,Longitude,Timezone,Daylight savings time flag
648,10001,5.845588,New York,NY,40.750742,-73.996530,-5.0,1.0
649,10002,5.068544,New York,NY,40.717040,-73.987000,-5.0,1.0
650,10003,7.640783,New York,NY,40.732509,-73.989350,-5.0,1.0
652,10005,6.469799,New York,NY,40.706019,-74.008580,-5.0,1.0
653,10006,5.413333,New York,NY,40.707904,-74.013420,-5.0,1.0
...,...,...,...,...,...,...,...,...
3008,14901,5.823712,Elmira,NY,42.091924,-76.792170,-5.0,1.0
3009,14902,7.070175,Elmira,NY,42.147285,-76.750888,-5.0,1.0
3010,14903,6.749054,Elmira,NY,42.129274,-76.844970,-5.0,1.0
3011,14904,5.899058,Elmira,NY,42.072024,-76.803930,-5.0,1.0


In [12]:

#Plot Heatmap for vehicle age by zip code
locations = vehicle_age_2009_df[["Latitude", "Longitude"]]
vehicle_age = vehicle_age_2009_df["vehicle_age"].astype(float)

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=vehicle_age, 
                                  dissipating=False, max_intensity=1,                                  
                                  point_radius=0.05)


# Add layer to the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…